In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from mtcnn.mtcnn import MTCNN

detector = MTCNN()
def cropFaces(img):
    
    # detect faces in the image
    faces = detector.detect_faces(img)
    #print(faces)
    if len(faces) == 0:
        return faces
    x, y, width, height = faces[0]['box']
    #img[y:y+height , x:x+width, :]
    return img[y:y+height, x:x+width, :]


In [2]:
# Set the paths to your training and test data directories
# The images should be in seperate folders labelled with their corresponding names
train_data_dir = r"C:\Users\jv258\Documents\Jupyter\Projects\Jodhpur\Images\Train" #Edit path to Training image
test_data_dir = r"C:\Users\jv258\Documents\Jupyter\Projects\Jodhpur\Images\Test" #Edit path to Test images

# Define the image dimensions and batch size
img_width, img_height = 150, 150
batch_size = 32

# Preprocess, Augment and Normalising the training data
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
train_generator = train_datagen.flow_from_directory(train_data_dir,
                                                    target_size=(img_width, img_height),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

# Preprocess the test data (no augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(test_data_dir,
                                                  target_size=(img_width, img_height),
                                                  batch_size=batch_size,
                                                  class_mode='categorical')

Found 75 images belonging to 5 classes.
Found 75 images belonging to 5 classes.


In [3]:
for images, labels in train_generator:
    print("Batch shape:", images.shape)
    print("Labels:", labels)
    break


Batch shape: (32, 150, 150, 3)
Labels: [[0. 0. 0. 0. 1.]
 [0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0.]]


In [4]:
# Define the number of classes
num_classes = int(labels[0].size)

# Create the CNN model
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())

model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [5]:
#the model is initialized to use the adam algorithm
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# It was noticed that the number of epochs had an effect on the model accuracy. But since 
# the accuracy did not show much deviation and reached a maximum after 50 epochs the epochs is set to 60
model.fit(train_generator,
          steps_per_epoch=len(train_generator),
          epochs=50,
          validation_data=test_generator,
          validation_steps=len(test_generator))

model.save('image_recognition_model.h5')

Epoch 1/50
3/3 [==============================] - 7s 2s/step - loss: 1.8257 - accuracy: 0.1600 - val_loss: 1.6230 - val_accuracy: 0.2000
Epoch 2/50
3/3 [==============================] - 3s 816ms/step - loss: 1.6235 - accuracy: 0.2000 - val_loss: 1.6116 - val_accuracy: 0.2267
Epoch 3/50
3/3 [==============================] - 2s 659ms/step - loss: 1.5947 - accuracy: 0.2800 - val_loss: 1.6118 - val_accuracy: 0.2000
Epoch 4/50
3/3 [==============================] - 3s 1s/step - loss: 1.6274 - accuracy: 0.1867 - val_loss: 1.6252 - val_accuracy: 0.2000
Epoch 5/50
3/3 [==============================] - 3s 830ms/step - loss: 1.5997 - accuracy: 0.2533 - val_loss: 1.5924 - val_accuracy: 0.2267
Epoch 6/50
3/3 [==============================] - 3s 1s/step - loss: 1.5709 - accuracy: 0.3467 - val_loss: 1.5789 - val_accuracy: 0.2133
Epoch 7/50
3/3 [==============================] - 2s 973ms/step - loss: 1.5880 - accuracy: 0.2267 - val_loss: 1.5893 - val_accuracy: 0.2667
Epoch 8/50
3/3 [=============

In [6]:
loaded_model = tf.keras.models.load_model('image_recognition_model.h5')

test_image_path = input('Enter the full path of the image without"":\n')

test_image = tf.keras.preprocessing.image.load_img(test_image_path, target_size=(img_width, img_height))
test_image = tf.keras.preprocessing.image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image = test_image / 255.0

predictions = loaded_model.predict(test_image)
predicted_label = np.argmax(predictions)

class_labels = train_generator.class_indices
person_label = {v: k for k, v in class_labels.items()}
predicted_person = person_label[predicted_label]

print('Predicted Person:', predicted_person)


Enter the full path of the image without"":
f


FileNotFoundError: [Errno 2] No such file or directory: 'f'

In [7]:
# The model has some inaccuracies which can be due to the algorithm
# or the limited number of training images

In [8]:
import os
import cv2

# Set the path to the folder containing the images
folder_path = r"C:\Users\jv258\Documents\Jupyter\Projects\Jodhpur\Images\Test\Keira Knightley"


for filename in os.listdir(folder_path):
        
    image_path = os.path.join(folder_path, filename)

        
    image = cv2.imread(image_path)
    
    image = cv2.resize(image, (150, 150))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)         
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = np.expand_dims(image, axis=0)
    image = image / 255.0
    
    pred = loaded_model.predict(image)
    
    pred_label = np.argmax(pred)
    class_labels = train_generator.class_indices
    person_label = {v: k for k, v in class_labels.items()}
    pred_person = person_label[pred_label]
        
    print('Predicted Person:', pred_person)



1/1 [==============================] - 0s 76ms/step
Predicted Person: Zac Efron
1/1 [==============================] - 0s 22ms/step
Predicted Person: Keira Knightley
1/1 [==============================] - 0s 22ms/step
Predicted Person: Tom Cruise
1/1 [==============================] - 0s 21ms/step
Predicted Person: Shahrukh Khan
1/1 [==============================] - 0s 23ms/step
Predicted Person: Keira Knightley
1/1 [==============================] - 0s 21ms/step
Predicted Person: Barack Obama
1/1 [==============================] - 0s 23ms/step
Predicted Person: Keira Knightley
1/1 [==============================] - 0s 7ms/step
Predicted Person: Tom Cruise
1/1 [==============================] - 0s 21ms/step
Predicted Person: Shahrukh Khan
1/1 [==============================] - 0s 18ms/step
Predicted Person: Zac Efron
1/1 [==============================] - 0s 22ms/step
Predicted Person: Keira Knightley
1/1 [==============================] - 0s 24ms/step
Predicted Person: Zac Efron
1/1 